<a href="https://colab.research.google.com/github/rahul-mahatdeveloper/alarm/blob/main/traveldemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_core langchain_groq langchain_community langchain langgraph
!pip install python-dotenv
!pip install --upgrade langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.2 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv

In [ ]:
GROQ_API_KEY="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"


In [ ]:
!pip install --upgrade langchain langchain_community

In [ ]:
def fetch_attractions(state):
    # Interests ko comma-separated string me convert karo
    interests_str = ', '.join(state['interests'])

    # Google Search Query Banao
    search_query = f"Best {interests_str} places to visit in {state['city']}"

    # Search Results Fetch Karo
    attractions = search.run(search_query)

    return attractions



In [ ]:
def fetch_attractions(state):
    # Interests ko comma-separated string me convert karo
    interests_str = ', '.join(state['interests'])

    # Google Search Query Banao
    search_query = f"Best {interests_str} places to visit in {state['city']}"

    # Search Results Fetch Karo
    attractions = search.run(search_query)

    return attractions


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")


In [ ]:

import os
import requests
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

def get_user_input(prompt: str) -> str:
    return input(f"{prompt}: ")

class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "conversation messages"]
    city: str
    interests: List[str]
    budget: Optional[str]
    duration: Optional[str]
    dietary_prefs: Optional[str]
    mobility: Optional[str]
    itinerary: str

def get_location(city):
    url = f"https://nominatim.openstreetmap.org/search?q={city}&format=json"
    response = requests.get(url)
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return float(data["lat"]), float(data["lon"])
    return None

def fetch_attractions(state):
    latlon = get_location(state['city'])
    if not latlon:
        return "City not found!"

    lat, lon = latlon
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node(around:10000,{lat},{lon})["tourism"];
    out;
    """
    response = requests.get(overpass_url, params={'data': query})

    if response.status_code == 200:
        data = response.json()
        places = [node['tags'].get('name', 'Unknown Place') for node in data.get("elements", []) if 'tags' in node]
        return places if places else ["No attractions found."]
    else:
        return "Error fetching data!"

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama3-70b-8192"
)

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a smart travel assistant. Generate a well-structured itinerary for {city} based on user details: interests={interests}, budget={budget}, duration={duration}, dietary preferences={dietary_prefs}, mobility={mobility}.")
])

def input_city(state: PlannerState) -> PlannerState:
    user_message = get_user_input("Enter the city you want to visit")
    return {**state, "city": user_message, "messages": state["messages"] + [HumanMessage(content=user_message)]}

def input_interest(state: PlannerState) -> PlannerState:
    user_message = get_user_input(f"Enter your interests for {state['city']} (comma-separated)")
    return {**state, "interests": [x.strip() for x in user_message.split(",")], "messages": state["messages"] + [HumanMessage(content=user_message)]}

def input_budget_duration(state: PlannerState) -> PlannerState:
    budget = get_user_input("Enter your budget (low, medium, high)")
    duration = get_user_input("Enter trip duration (e.g., 2 days, weekend, etc.)")
    return {**state, "budget": budget, "duration": duration, "messages": state["messages"] + [HumanMessage(content=f"Budget: {budget}, Duration: {duration}")]}

def input_preferences(state: PlannerState) -> PlannerState:
    dietary_prefs = get_user_input("Any dietary preferences? (e.g., vegetarian, halal, no preference)")
    mobility = get_user_input("Any mobility concerns? (e.g., need wheelchair access, can walk long distances, no preference)")
    return {**state, "dietary_prefs": dietary_prefs, "mobility": mobility, "messages": state["messages"] + [HumanMessage(content=f"Diet: {dietary_prefs}, Mobility: {mobility}")]}

def create_itinerary(state: PlannerState) -> PlannerState:
    attractions = fetch_attractions(state)
    attraction_list = "\n".join(attractions)

    response = llm.invoke(itinerary_prompt.format_messages(
        city=state['city'], interests=', '.join(state['interests']),
        budget=state['budget'], duration=state['duration'],
        dietary_prefs=state['dietary_prefs'], mobility=state['mobility']
    ))

    print("\nGenerated Itinerary:\n", response.content)
    print("\nTop Attractions:\n", attraction_list)

    return {**state, "itinerary": response.content, "messages": state["messages"] + [AIMessage(content=response.content)]}

workflow = StateGraph(PlannerState)
workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("input_budget_duration", input_budget_duration)
workflow.add_node("input_preferences", input_preferences)
workflow.add_node("create_itinerary", create_itinerary)
workflow.set_entry_point("input_city")
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "input_budget_duration")
workflow.add_edge("input_budget_duration", "input_preferences")
workflow.add_edge("input_preferences", "create_itinerary")
workflow.add_edge("create_itinerary", END)
app = workflow.compile()

def travel_planner(user_request: str):
    print(f"Request: {user_request}\n")
    state = {"messages": [HumanMessage(content=user_request)], "city": "", "interests": [], "budget": None, "duration": None, "dietary_prefs": None, "mobility": None, "itinerary": ""}
    for output in app.stream(state):
        pass

travel_planner("Plan a trip for me")


Request: Plan a trip for me

Enter the city you want to visit: belpada
Enter your interests for belpada (comma-separated): roaming
Enter your budget (low, medium, high): low
Enter trip duration (e.g., 2 days, weekend, etc.): weekend
Any dietary preferences? (e.g., vegetarian, halal, no preference): halal
Any mobility concerns? (e.g., need wheelchair access, can walk long distances, no preference): no preference

Generated Itinerary:
 Belpada! A charming town in Odisha, India, surrounded by lush green forests and scenic beauty. I've crafted a weekend itinerary for you, taking into account your interests, budget, dietary preferences, and mobility requirements. Here's a fun-filled and affordable trip plan for you:

**Itinerary:**

**Day 1 (Saturday):**

* **Morning (9:00 am - 10:30 am):** Start your day with a delicious Halal breakfast at **Al-Falah Restaurant**, a popular eatery in Belpada serving mouth-watering dishes at an affordable price (₹100 - ₹200 per person).
* **11:00 am - 1:00 

In [2]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00


In [4]:
!pip install langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 1.8 MB/s eta 0:00:00


In [5]:
import gradio as gr
import requests
from typing import TypedDict, List, Optional
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Fetch location from OpenStreetMap
def get_location(city):
    url = f"https://nominatim.openstreetmap.org/search?q={city}&format=json"
    response = requests.get(url)
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return float(data["lat"]), float(data["lon"])
    return None

# Fetch attractions from Overpass API
def fetch_attractions(city):
    latlon = get_location(city)
    if not latlon:
        return ["City not found!"]

    lat, lon = latlon
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node(around:10000,{lat},{lon})["tourism"];
    out;
    """
    response = requests.get(overpass_url, params={'data': query})

    if response.status_code == 200:
        data = response.json()
        places = [node['tags'].get('name', 'Unknown Place') for node in data.get("elements", []) if 'tags' in node]
        return places if places else ["No attractions found."]
    else:
        return ["Error fetching data!"]

# Initialize LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama3-70b-8192"
)

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a smart travel assistant. Generate a well-structured itinerary for {city} based on user details: interests={interests}, budget={budget}, duration={duration}, dietary preferences={dietary_prefs}, mobility={mobility}.")
])

# Function to generate itinerary
def generate_itinerary(city, interests, budget, duration, dietary_prefs, mobility):
    interests_list = interests.split(",") if interests else []
    attractions = fetch_attractions(city)
    attraction_list = "\n".join(attractions)

    response = llm.invoke(itinerary_prompt.format_messages(
        city=city, interests=', '.join(interests_list),
        budget=budget, duration=duration,
        dietary_prefs=dietary_prefs, mobility=mobility
    ))

    itinerary = response.content

    return itinerary, attraction_list

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("## ✈️ Smart Travel Planner 🌍")
    gr.Markdown("Plan your trip dynamically with AI! Enter your details below and get a personalized itinerary.")

    with gr.Row():
        city = gr.Textbox(label="🌆 City to Visit", placeholder="Enter city name")
        duration = gr.Textbox(label="🗓 Duration", placeholder="e.g., 2 days, weekend")

    with gr.Row():
        budget = gr.Radio(["Low", "Medium", "High"], label="💰 Budget")
        dietary_prefs = gr.Textbox(label="🍽 Dietary Preferences", placeholder="e.g., Vegetarian, No preference")

    with gr.Row():
        interests = gr.Textbox(label="🎯 Interests", placeholder="e.g., Museums, Beaches, Adventure")
        mobility = gr.Textbox(label="♿ Mobility Needs", placeholder="e.g., Wheelchair access, No preference")

    generate_button = gr.Button("🚀 Generate Itinerary")

    itinerary_output = gr.Textbox(label="🗺 Your Itinerary", interactive=False)
    attractions_output = gr.Textbox(label="🏛 Top Attractions", interactive=False)

    generate_button.click(
        generate_itinerary,
        inputs=[city, interests, budget, duration, dietary_prefs, mobility],
        outputs=[itinerary_output, attractions_output]
    )

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73b977b5669b842fec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
